In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
from citipy import citipy

In [2]:
csv_path = "RawData/Zip_Zhvi_SingleFamilyResidence.csv"

In [3]:
singlefam_data = pd.io.parsers.read_csv(csv_path, encoding='ISO-8859-1',dtype={'RegionName': 'str'})
singlefam_data['RegionName'] = singlefam_data['RegionName'].apply(lambda x: x.zfill(5))
singlefam_data.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2016-01,2016-02,2016-03,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,1416633.0,1427321.0,1441979.0,...,1380332,1386270,1394397,1404225,1406599,1399918,1380178,1358401,1350481,1345845
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,907374.0,921322.0,930106.0,...,966123,967557,965155,960225,956709,953095,950684,948136,946838,945928
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,1556960.0,1598874.0,1627446.0,...,1520816,1526676,1525174,1516721,1504180,1492086,1480934,1468075,1454870,1438313
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,336604.0,335648.0,334774.0,...,335789,335035,334542,334176,334363,334127,334458,334460,334679,334309
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,1114867.0,1131654.0,1142148.0,...,1198898,1203423,1204840,1199747,1194591,1188702,1183500,1179024,1175407,1174008


In [4]:
singlefam_data.columns.values.tolist()

['RegionID',
 'RegionName',
 'City',
 'State',
 'Metro',
 'CountyName',
 'SizeRank',
 '2016-01',
 '2016-02',
 '2016-03',
 '2016-04',
 '2016-05',
 '2016-06',
 '2016-07',
 '2016-08',
 '2016-09',
 '2016-10',
 '2016-11',
 '2016-12',
 '2017-01',
 '2017-02',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11',
 '2019-12']

In [5]:
#Drop unnecessary columns, will analyze 2018 data
zillow_2019_data = singlefam_data.drop(['2016-01','2016-02','2016-03','2016-04','2016-05',
                                        '2016-06','2016-07','2016-08','2016-09','2016-10',
                                        '2016-11','2016-12','2017-01','2017-02','2017-03',
                                        '2017-04','2017-05','2017-06','2017-07','2017-08',
                                        '2017-09','2017-10','2017-11','2017-12','2019-01',
                                        '2019-02','2019-03','2019-04','2019-05','2019-06',
                                        '2019-07','2019-08','2019-09','2019-10','2019-11','2019-12'], axis=1)
zillow_2019_data.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,1468482,1461572,1454050,1454111,1451781,1449835,1450723,1440314,1433973,1429073,1428637,1425987
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,941705,953981,959260,960954,959673,959082,954912,950660,948543,948906,951721,954743
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,1531018,1537936,1551681,1568820,1573613,1578249,1582848,1583553,1580068,1573052,1563902,1550264
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,331699,332214,332264,332737,333805,334293,333806,333609,333839,334129,334753,334497
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,1186741,1197817,1201566,1204015,1204371,1201598,1195307,1189358,1186838,1185950,1187917,1190385


In [6]:
zillow_2019_data.dtypes

RegionID       int64
RegionName    object
City          object
State         object
Metro         object
CountyName    object
SizeRank       int64
2018-01        int64
2018-02        int64
2018-03        int64
2018-04        int64
2018-05        int64
2018-06        int64
2018-07        int64
2018-08        int64
2018-09        int64
2018-10        int64
2018-11        int64
2018-12        int64
dtype: object

In [7]:
zillow_2019_data['2018_AvgHomeValue'] = zillow_2019_data[['2018-01','2018-02','2018-03','2018-04','2018-05','2018-06',
                                                    '2018-07','2018-08','2018-09','2018-10',
                                                    '2018-11','2018-12']].sum(axis=1)/12

zillow_2019_data['2018_AvgHomeValue'] = zillow_2019_data['2018_AvgHomeValue'].astype(np.int32)
zillow_2019_data.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2018_AvgHomeValue
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,1468482,1461572,1454050,1454111,1451781,1449835,1450723,1440314,1433973,1429073,1428637,1425987,1445711
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,941705,953981,959260,960954,959673,959082,954912,950660,948543,948906,951721,954743,953678
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,1531018,1537936,1551681,1568820,1573613,1578249,1582848,1583553,1580068,1573052,1563902,1550264,1564583
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,331699,332214,332264,332737,333805,334293,333806,333609,333839,334129,334753,334497,333470
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,1186741,1197817,1201566,1204015,1204371,1201598,1195307,1189358,1186838,1185950,1187917,1190385,1194321


In [8]:
avg_2019_valdf=zillow_2019_data.drop(['2018-01','2018-02','2018-03','2018-04','2018-05','2018-06',
                       '2018-07','2018-08','2018-09','2018-10','2018-11',
                                      '2018-12', 'SizeRank'], axis=1).rename(columns={"RegionName": "ZipCode"})
avg_2019_valdf.head()

,RegionID,ZipCode,City,State,Metro,CountyName,2018_AvgHomeValue
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1445711
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,953678
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,1564583
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,333470
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,1194321


In [9]:
#zip_group_avg = avg_2019_valdf.groupby(['ZipCode'])

In [10]:
zip_group_avg = avg_2019_valdf.groupby(['ZipCode','City','State','Metro','CountyName'])['2018_AvgHomeValue'].mean().reset_index()

zip_group_avg.head()

,ZipCode,City,State,Metro,CountyName,2018_AvgHomeValue
0,00501,Holtsville,NY,New York-Newark-Jersey City,Suffolk County,94165
1,00544,Simsboro,LA,Ruston,Lincoln Parish,87623
2,00602,Auburn,AL,Auburn-Opelika,Lee County,68143
3,00603,Aguadilla,AL,Dothan,Houston County,77791
4,00604,Aguadilla,AL,Dothan,Houston County,63962


In [11]:
zip_group_avg_sorted = zip_group_avg.sort_values(by='2018_AvgHomeValue', ascending=False).reset_index()
zip_group_avg_sorted.head()

,index,ZipCode,City,State,Metro,CountyName,2018_AvgHomeValue
0,2066,10065,New York,NY,New York-Newark-Jersey City,New York County,8779974
1,2068,10075,New York,NY,New York-Newark-Jersey City,New York County,7859110
2,2045,10021,New York,NY,New York-Newark-Jersey City,New York County,7397214
3,2069,10128,New York,NY,New York-Newark-Jersey City,New York County,5937647
4,2039,10014,New York,NY,New York-Newark-Jersey City,New York County,5687485


In [12]:
clean_2019avgprice_zip_df = zip_group_avg_sorted.drop(['index'],axis=1).head(50)
clean_2019avgprice_zip_df

,ZipCode,City,State,Metro,CountyName,2018_AvgHomeValue
0,10065,New York,NY,New York-Newark-Jersey City,New York County,8779974
1,10075,New York,NY,New York-Newark-Jersey City,New York County,7859110
2,10021,New York,NY,New York-Newark-Jersey City,New York County,7397214
3,10128,New York,NY,New York-Newark-Jersey City,New York County,5937647
4,10014,New York,NY,New York-Newark-Jersey City,New York County,5687485
5,94027,Atherton,CA,San Francisco-Oakland-Hayward,San Mateo County,5320040
6,81611,Aspen,CO,Glenwood Springs,Pitkin County,5220188
7,02108,Boston,MA,Boston-Cambridge-Newton,Suffolk County,4818923
8,33480,Palm Beach,FL,Miami-Fort Lauderdale-West Palm Beach,Palm Beach County,4425368
9,10013,New York,NY,New York-Newark-Jersey City,New York County,4402882


In [13]:
#export Dataframe to CSV - add to final_data directory
clean_2019avgprice_zip_df.to_csv("CleanData/Zillow_2018avghomeval.csv",index=False)